# How to run a simple variational algorithm with qoqo

This example notebook is designed to show how to run a very simple variational algorithm with qoqo.
The variational algorithm will be a very simple example of a Variational Hamiltonian Ansatz (VHA), the code does not aim to get the best result possible but to show a very simple example.

For detailed discussions of variational algorithms, VHA and different variants of these algorithms see the literature (e.g. http://arxiv.org/abs/1304.3061, http://arxiv.org/abs/1509.04279).


## Example: VHA for spins using qoqo

The goal of a variational algorithm is simple: finding a good approximation of the ground state of a physical system defined by a Hamiltonian H by minimizing the expectation value of H with respect to a set of trial states $| \psi (\vec{\theta} )>$.
The optimization is carried out by optimizing the classical parameters $\vec{\theta}$ defining the trial states.  
By definition the ground state is the state with the lowest energy, so this approach will find the ground state if the ground-state is in the set of possible trial states and the classical optimizer is successfull.  

The trial states are prepared by applying a set of unitary transformations to an initial state
$$
| \psi (\vec{\theta}) > = \prod_j U_j (\vec{\theta}) | \psi_{\textrm{init}} >.
$$
In a VHA the ansatz is to assume that the Hamiltonian can be separated into partial Hamiltonians
$$
H = \sum_{\alpha} H_{\alpha}
$$
and use the time evolution under these partial Hamiltonians as the ansatz for the unitary transformations
$$
| \psi (\vec{\theta}) > = \prod_k^{N} \prod_{\alpha} \exp(-i \theta_{k,\alpha} H_{\alpha}) | \psi_{\textrm{init}}>,
$$
where N is the number of iterations of the pseudo time evolution and $(\theta_{k,\alpha})$ is the variational pseudo time.

Here we use as a sample Hamiltonian a one-dimensional spin chain with three sites
$$
H = H_0 + H_1 + H_2
$$
where $H_0$ is the magnetic onsite energy
$$
H_0 = B \left(\sigma^z_0 + \sigma^z_1 + \sigma^z_0\right),
$$
$H_1$ is the hopping between even and odd sites
$$
H_1 = t \sigma^x_0\sigma^x_1,
$$
and $H_2$ is the hopping between odd and even sites
$$
H_2 = t \sigma^x_1\sigma^x_2.
$$


### The VHA example consists of the following steps:

1. Define a circuit to initialize the initial state.
2. Define a time evolution circuit that contains the variational parametere $\vec{\theta}$ as symbolic parameters.
3. Define basis rotation (BR) measurement circuits and the measurement information containing t and B to measure the expectation value of $H$. 
4. Combine the parts in a qoqo quantum program that can be called with the free parameters and directly return the expectation values.
5. Use the compact quantum program to optimize the free parameters $\vec{\theta}$.

Additionally, an exact solution of the Hamiltonian is presented at the end to compare the exact results with the calculated solution.

In [32]:
# import standard python components
import numpy as np
import scipy.sparse as sp
import scipy.optimize as so
from typing import List

# import circuit operation modules from qoqo
from qoqo import Circuit
from qoqo import operations as ops
from hqsbase.calculator import CalculatorFloat

# simulation and measurement of the circuit is handled by the QuEST interface
from qoqo_quest import Backend

# import modules and classes from qoqo for measuring observables
from qoqo.measurements import BasisRotationInput, BasisRotation
from qoqo import QuantumProgram

In [33]:
# variables
number_measurements = 10000
number_qubits = 3


# parameters
magnetic_field = 1.0
hopping_parameter = 3.0

### 1. Initialization of the state vector by qoqo

In principle the initial state $\left|\psi_{\textrm{init}}\right>$ has to be prepared with a full quantum circuit (e.g. http://arxiv.org/abs/1711.05395), since we want to keep the example simple we will use a "cheated" initial state instead.

When using a simulator backend one can use a PRAGMA operation (qoqo.operations.pragma_operations.PragmaSetStateVector(vector)) to "cheat" and directly set the state vector on the simulator. 
The n-th entry in the state vector corresponds to the basis state $|b(n,2) b(n,1) b(b,0)>$ where $b(n,k)$ gives the k-th enty of the binary representation of n.
$$
0 \leftrightarrow |000>
$$
$$
1 \leftrightarrow |001>
$$
$$
2 \leftrightarrow |010>
$$
and so on.

We choose a starting vector that is 50% in the single excitation subspace and 50% fully occupied 
$$
|\psi_{\textrm{init}}> =  \frac{1}{\sqrt{6}}|001> + \frac{1}{\sqrt{6}} |010> + \frac{1}{\sqrt{6}} |100> + \frac{1}{\sqrt{2}} |111>.
$$
We do not include extra terms to change the number of excitations in the VHA ansatz. Choosing a good initial guess for the number of excitations helps with convergence. For VHA variations that automatically derive the right number of excitations see for example https://doi.org/10.1088/2058-9565/abe568.

In [34]:
initial_vector_array = np.array([0.0, 1/np.sqrt(6), 1/np.sqrt(6),0.0 ,1/np.sqrt(6), 0.0, 0.0, 1/np.sqrt(2)]) 

circuit_init = Circuit()

circuit_init += ops.PragmaSetStateVector(statevector=initial_vector_array)

print('Step 1: Constructed initialization circuit.')
print('        Number of qubits in the system: ', number_qubits, '.')

Step 1: Constructed initialization circuit.
        Number of qubits in the system:  3 .


### 2. Unitary time evolution

We construct circuits that apply time evolution under the even and odd hopping Hamiltonians and under the magnetic field using variables t (hopping_parameter) and B (magnetic_field).  
For each iteration of the evolution we get free symbolic parameters theta_even_i, theta_odd_i and theta_z_i. 

In [35]:
# variational evolution for the hopping terms
def create_even_hopping_circuit(thetasymb: CalculatorFloat) -> Circuit:
    """
    Create circuit for evolution under even-to-odd hopping.
    
    Args:
        thetasymb: symbolic parameter 'theta' of the even-to-odd time evolution.
    """
    circuit = Circuit()

    # Decomposition of the \sigma^x\sigma^x interaction between two spins in CNOT and 
    # Rotation gates
    for k in range(0, number_qubits - 1, 2):
        circuit += ops.Hadamard(qubit=k)
        circuit += ops.Hadamard(qubit=k+1)
        circuit += ops.CNOT(control=k + 1, target=k)
        circuit += ops.RotateZ(qubit=k, theta=thetasymb * hopping_parameter)
        circuit += ops.CNOT(control=k + 1, target=k)
        circuit += ops.Hadamard(qubit=k)
        circuit += ops.Hadamard(qubit=k+1)
    
    return circuit

def create_odd_hopping_circuit(thetasymb: CalculatorFloat) -> Circuit:
    """
    Create circuit for evolution under odd-to-even hopping.
    
    Args:
        thetasymb: symbolic parameter 'theta' of the odd-to-even time evolution.
    """
    circuit = Circuit()

    # Decomposition of the \sigma^x\sigma^x interaction between two spins in CNOT and 
    # Rotation gates
    for k in range(1, number_qubits - 1, 2):
        circuit += ops.Hadamard(qubit=k)
        circuit += ops.Hadamard(qubit=k+1)
        circuit += ops.CNOT(control=k + 1, target=k)
        circuit += ops.RotateZ(qubit=k, theta=thetasymb * hopping_parameter)
        circuit += ops.CNOT(control=k + 1, target=k)
        circuit += ops.Hadamard(qubit=k)
        circuit += ops.Hadamard(qubit=k+1)
    # Periodic boundary conditions
    circuit += ops.Hadamard(qubit=number_qubits - 1)
    circuit += ops.Hadamard(qubit=0)
    circuit += ops.CNOT(control=0, target=number_qubits - 1)
    circuit += ops.RotateZ(qubit=number_qubits - 1, theta=thetasymb * hopping_parameter)
    circuit += ops.CNOT(control=0, target=number_qubits - 1)
    circuit += ops.Hadamard(qubit=number_qubits - 1)
    circuit += ops.Hadamard(qubit=0)

    return circuit


# variational evolution for the magnetic term
def create_magnetic_field_circuit(thetasymb: CalculatorFloat) -> Circuit:
    """
    Create circuit for evolution under magnetic field.

    Args:
        thetasymb: symbolic parameter 'theta' for the z-rotation.
    """
    circuit = Circuit()
    for i in range(number_qubits):
        circuit += ops.RotateZ(qubit=i, theta=thetasymb * magnetic_field)
    return circuit


def create_evolution_circuit(
        iter_evolution: int,
        ) -> Circuit:
    """
    Construct the circuit for the unitary evolution.

    Args:
        iter_evolution: number of iterations of evolution, minimum 1.
    """
    # here: theta_even_i, theta_odd_i and theta_z_i are symbolic parameters (free variational parameters)
    circuit = Circuit()
    for i in range(iter_evolution):
        circuit += create_even_hopping_circuit(CalculatorFloat('theta_even_' + str(i)))
        circuit += create_odd_hopping_circuit(CalculatorFloat('theta_odd_' + str(i)))
        circuit += create_magnetic_field_circuit(CalculatorFloat('theta_z_' + str(i)))
    return circuit


# In order to achieve better minimization results we default to several iterations of (pseudo) time-evolution
iter_evolution = 4
# Construct the evolution
circuit_evolution = create_evolution_circuit(iter_evolution)

print('Step 2: Constructed evolution circuit.')
print(circuit_evolution)

Step 2: Constructed evolution circuit.
Hadamard(Hadamard { qubit: 0 })
Hadamard(Hadamard { qubit: 1 })
CNOT(CNOT { control: 1, target: 0 })
RotateZ(RotateZ { qubit: 0, theta: Str("(theta_even_0 * 3e0)") })
CNOT(CNOT { control: 1, target: 0 })
Hadamard(Hadamard { qubit: 0 })
Hadamard(Hadamard { qubit: 1 })
Hadamard(Hadamard { qubit: 1 })
Hadamard(Hadamard { qubit: 2 })
CNOT(CNOT { control: 2, target: 1 })
RotateZ(RotateZ { qubit: 1, theta: Str("(theta_odd_0 * 3e0)") })
CNOT(CNOT { control: 2, target: 1 })
Hadamard(Hadamard { qubit: 1 })
Hadamard(Hadamard { qubit: 2 })
Hadamard(Hadamard { qubit: 2 })
Hadamard(Hadamard { qubit: 0 })
CNOT(CNOT { control: 0, target: 2 })
RotateZ(RotateZ { qubit: 2, theta: Str("(theta_odd_0 * 3e0)") })
CNOT(CNOT { control: 0, target: 2 })
Hadamard(Hadamard { qubit: 2 })
Hadamard(Hadamard { qubit: 0 })
RotateZ(RotateZ { qubit: 0, theta: Str("theta_z_0") })
RotateZ(RotateZ { qubit: 1, theta: Str("theta_z_0") })
RotateZ(RotateZ { qubit: 2, theta: Str("theta_z_0

### 3. Basis rotation (BR) measurement to get the expectation values

We construct the basis rotation circuits for the measurement of the separate parts of the Hamiltonian.
The magnetic field part of the Hamiltonian contains only $\sigma^z$ operators and can be measured in the z-basis of all qubits. The hopping parts of the Hamiltonian contain only products of $\sigma^x$ operators and can be measured in the x-basis of all qubits. For more information on the basis rotation measurement see the "Introduction to qoqo" example.  

After constructing the measurement circuit and the measurement information we combine everything into one qoqo measurement.

In [36]:
# Setting up two basis rotation measurement circuits since we need to measure in two different bases
x_basis_measurement_circuit = Circuit()
z_basis_measurement_circuit = Circuit()

# The qoqo operation 'DefinitionBit()' defines the classical bit registers in the circuit used to store the qubit readout
x_basis_measurement_circuit += ops.DefinitionBit(
    name='ro_x',  length=number_qubits, is_output=True) # parameter 'length' is the number of qubits to be measured
z_basis_measurement_circuit += ops.DefinitionBit(
    name='ro_z', length=number_qubits, is_output=True) # parameter 'length' is the number of qubits to be measured

# Basis rotation with the Hadamard gate: Bring all qubits into z-basis
for i in range(number_qubits):
    x_basis_measurement_circuit += ops.Hadamard(qubit=i)
    
# Add measurement operation to all circuits to write the measured values into the classical registers
z_basis_measurement_circuit += ops.PragmaRepeatedMeasurement(
    readout='ro_z', number_measurements=number_measurements, qubit_mapping=None)
x_basis_measurement_circuit += ops.PragmaRepeatedMeasurement(
    readout='ro_x', number_measurements=number_measurements, qubit_mapping=None)

# Setting up measurement input determining which expectation values of PauliProducts are measured from which circuit
# and how they are combined linearly
measurement_input = BasisRotationInput(number_qubits=number_qubits, use_flipped_measurement=False)
# Adding the measured Pauli products
index0 = measurement_input.add_pauli_product(readout="ro_z", pauli_product_mask=[0])
index1 = measurement_input.add_pauli_product(readout="ro_z", pauli_product_mask=[1])
index2 = measurement_input.add_pauli_product(readout="ro_z", pauli_product_mask=[2])
index3 = measurement_input.add_pauli_product(readout="ro_x", pauli_product_mask=[0, 1])
index4 = measurement_input.add_pauli_product(readout="ro_x", pauli_product_mask=[1, 2])
index5 = measurement_input.add_pauli_product(readout="ro_x", pauli_product_mask=[0, 2])


# Adding the linear combinations of Pauli products that give the expectation values
measurement_input.add_linear_exp_val(
    name="energy",
    linear={0:magnetic_field, 1:magnetic_field, 2:magnetic_field,
            3:hopping_parameter, 4:hopping_parameter, 5:hopping_parameter
           })

print('Step 3: Measurement circuits constructed.')
print('z-basis: ')
print(z_basis_measurement_circuit)
print('x-basis: ')
print(x_basis_measurement_circuit)

Step 3: Measurement circuits constructed.
z-basis: 
DefinitionBit(DefinitionBit { name: "ro_z", length: 3, is_output: true })
PragmaRepeatedMeasurement(PragmaRepeatedMeasurement { readout: "ro_z", number_measurements: 10000, qubit_mapping: None })

x-basis: 
DefinitionBit(DefinitionBit { name: "ro_x", length: 3, is_output: true })
Hadamard(Hadamard { qubit: 0 })
Hadamard(Hadamard { qubit: 1 })
Hadamard(Hadamard { qubit: 2 })
PragmaRepeatedMeasurement(PragmaRepeatedMeasurement { readout: "ro_x", number_measurements: 10000, qubit_mapping: None })



### 4. Combining the parts to QuantumProgram

To execute the optimization we combine all the constructed circuits and put them in one qoqo quantum programm. For the QuantumProgram we only need to provide the free parameters and get back the measured expectation values.

In [37]:
# Construct basis rotation measurement to get the expectation values
measurement = BasisRotation(
  input=measurement_input, 
  circuits=[
    circuit_init + circuit_evolution + z_basis_measurement_circuit,
    circuit_init + circuit_evolution + x_basis_measurement_circuit
  ],
  constant_circuit=None,
)  

# one needs to define a backend where the program is executed
backend = Backend(number_qubits=number_qubits)
# QuantumProgram takes the prepared list of circuits and the list of free parameter names (the symbolic values in the circuit)
program = QuantumProgram(
  measurement=measurement,
  input_parameter_names=[
    'theta_even_0','theta_odd_0', 'theta_z_0',
    'theta_even_1','theta_odd_1', 'theta_z_1',
    'theta_even_2','theta_odd_2', 'theta_z_2',
    'theta_even_3','theta_odd_3', 'theta_z_3',
  ]
)

print('Step 4: QuantumProgram constructed.')

Step 4: QuantumProgram constructed.


### 5. Optimization of free parameters
Minimiization routine to optimize the free parameters. This quantum program has 12 free parameters: theta_even, theta_odd and theta_z for each of the 4 iterations of evolution.

In [38]:
# 
def do_measurement(theta: List[float]) -> float:
    """A helper function wrapping QuantumProgramin the form required by the scipy optimizer. 

    Args:
        theta: List of optimized parameters.

    Returns:
        Cost function (energy expectation value).
    """
    exp_val = program.run(backend, theta)
    return np.real(exp_val['energy'])

# standard scipy optimization routine
final_result = so.minimize(
    fun=do_measurement,  # function to minimize
    x0=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],  # array of initial values for the free parameters theta
    method='COBYLA',
)

print('==> Optimized parameters theta: ', final_result.x, '.')
print('==> Calculated approximate Energy value: ', final_result.fun, '.')

==> Optimized parameters theta:  [ 0.43508345  0.02350965  1.16949616 -0.38445064 -0.43180257  0.99078984
 -0.04818821 -0.11615002 -0.39790998 -0.01509842 -0.04126061 -0.33272809] .
==> Calculated approximate Energy value:  -5.225199999999999 .


## PART II: Compare the calculated (approximate) result to the exact classical solution

Here we present an exact solution of the sample Hamiltonian to compare the exact results to the calculated solution of the VHA method.

In [39]:
# fist, define Hamiltonian
# pauli matrices
sigmax = sp.csr_matrix([[0.0, 1.0], [1.0, 0.0]])
sigmay = sp.csr_matrix([[0.0, 1.0j], [-1.0j, 0.0]])   # not required in this example
sigmaz = sp.csr_matrix([[1.0, 0.0], [0.0, -1.0]])
# identity matrix
identity = sp.csr_matrix([[1.0, 0.0], [0.0, 1.0]])

# magnetic term for 3 qubits
H_magnetic = magnetic_field * (
    sp.kron(sp.kron(sigmaz, identity), identity)
    + sp.kron(sp.kron(identity, sigmaz), identity)
    + sp.kron(sp.kron(identity, identity), sigmaz))
# hopping term for 3 qubits
H_hopping = hopping_parameter * (
    sp.kron(sp.kron(sigmax, sigmax), identity)
    + sp.kron(sp.kron(identity, sigmax), sigmax)
    + sp.kron(sp.kron(sigmax, identity), sigmax)
)
# total Hamiltonian
H = H_hopping + H_magnetic
print(H)

# diagonalize the Hamiltonian H, calculate eigenvalues and eigenvectors
print('Step 4: Diagonalization of the classical Hamiltonian.')
(eigenvalues, eigenvectors) = sp.linalg.eigsh(H, which='SA')    # sorted
print('==> Energy of the ground state: ', "%.4f" % eigenvalues.real[0], '.')

# final print-out
delta = final_result.fun - eigenvalues.real[0]
print('Difference between VHA result and exact result: ', "%.4f" % delta, '.')

Step 4: Diagonalization of the classical Hamiltonian.
==> Energy of the ground state:  -5.2111 .
Difference between VHA result and exact result:  -0.0141 .
